<a href="https://colab.research.google.com/github/phonhay103/anything/blob/Books/SPP_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [6]:
class AdaptivePooling2D(tf.keras.layers.Layer):
    def __init__ (self, reduce_function, output_size):
        self.reduce_function = reduce_function
        self.output_size = output_size
        super().__init__()

    def call(self, inputs):
        h_bins = self.output_size[0]
        w_bins = self.output_size[1]

        split_cols = tf.split(inputs, h_bins, axis=1)
        split_cols = tf.stack(split_cols, axis=1)
        split_rows = tf.split(split_cols, w_bins, axis=3)
        split_rows = tf.stack(split_rows, axis=3)
        return self.recude_function(split_rows, axis=[2, 4])

    def compute_output_shape(self, input_shape):
        input_shape = tf.TensorShape(input_shape).as_list()
        return tf.TensorShape([input_shape[0], self.output_size[0], self.output_size[1], input_shape[3],])

In [10]:
class AdaptiveAveragePooling2D(AdaptivePooling2D):
    def __init__ (self, output_size):
        super().__init__(tf.reduce_mean, output_size)

class AdaptiveMaxPooling2D(AdaptivePooling2D):
    def __init__ (self, output_size):
        super().__init__(tf.reduce_max, output_size)

In [17]:
class SpatialPyramidPooling2D(tf.keras.layers.Layer):
    def __init__ (self, bins):
        self.bins = bins
        self.pool_layers = [AdaptiveAveragePooling2D(bin) for bin in self.bins]
        super().__init__()

    def call(self, inputs):
        dynamic_input_shape = tf.shape(inputs)
        outputs = []
        for index, bin in enumerate(self.bins):
            new_height = dynamic_input_shape[1] - dynamic_input_shape[1] % bin[0]
            new_width = dynamic_input_shape[2] - dynamic_input_shape[2] % bin[1]
            new_shape = inputs[:, :new_height, :new_width, :]

            output = self.pool_layers[index](new_shape)
            output = tf.reshape(output, [dynamic_input_shape[0], bin[0] * bin[1], inputs.shape[-1]])
            outputs.append(output)

        return tf.concat(outputs, axis=1)

    def compute_output_shape(self, input_shape):
        pooled_shape = 0
        for bin in self.bins:
            pooled_shape += tf.reduce_prod(bin)

        return tf.TensorShape([input_shape[0], pooled_shape, input_shape[-1]])